In [1]:
import itertools
import pandas as pd
import numpy as np

/home/james/miniconda3/envs/vivarium/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/james/miniconda3/envs/vivarium/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [13]:
# Input Data
YEAR_START = 2011
AGE_GROUP_END = 110
YEAR_END = 2026 + AGE_GROUP_END

def get_annual_percent_change():
    return pd.DataFrame(
        list(itertools.product(range(YEAR_START, YEAR_END + 1), 
                               range(AGE_GROUP_END + 1), 
                               ['male', 'female'], 
                               [0])),
                        columns=['year', 'age', 'sex', 'value']
    )

def get_base_mortality():
    df = pd.read_csv('/home/james/Downloads/inputs_take2.csv')
    df['year'] = YEAR_START
    df = df.rename(columns={'mortality per 1\nrate': 'rate'})
    acmr = df[['age', 'sex', 'year', 'rate']]
    return acmr.sort_values(by=['year', 'age', 'sex']).reset_index(drop=True)

def get_base_population():
    df = pd.read_csv('/home/james/Downloads/inputs_take1.csv')
    df['year'] = YEAR_START    
    pop = df[['age', 'sex', 'year', 'pop_avg_5yr']]
    for age_group in range(0, AGE_GROUP_END, 5):
        age_at = age_group + 2
        for age in range(age_group, age_group + 5):
            pop.loc[pop.age == age, 'pop_avg_5yr'] = pop.loc[pop.age == age_at, 'pop_avg_5yr'].values
    pop.loc[pop.age == AGE_GROUP_END, 'pop_avg_5yr'] = 0
    pop = pop.rename(columns={'pop_avg_5yr': 'population'})
    return pop.sort_values(by=['year', 'age', 'sex']).reset_index(drop=True)


In [41]:
def get_all_cause_mortality():
    all_apc = get_annual_percent_change()
    current_mortality = get_base_mortality().set_index(['age', 'sex', 'year'])
    
    data = [current_mortality]
    for year in range(YEAR_START, YEAR_END):
        apc = all_apc[all_apc.year == year].set_index(['age', 'sex', 'year'])
        new_mortality = pd.DataFrame({'rate': current_mortality.rate * (1 + apc.value/100)}).reset_index()
        new_mortality['year'] = year + 1
        new_mortality = new_mortality.set_index(['age', 'sex', 'year'])
        data.append(new_mortality)
        current_mortality = new_mortality
        
    return pd.concat(data).reset_index().sort_values(['year', 'age', 'sex']).reset_index(drop=True)
    

def get_probability_of_death():
    acmr = get_all_cause_mortality().set_index(['age', 'sex', 'year'])
    p = (1 -  np.exp(-acmr)).reset_index()
    return p.rename(columns={'rate': 'probability'}).sort_values(by=['year', 'age', 'sex']).reset_index(drop=True)


def get_population():
    current_pop = get_base_population()
    all_p = get_probability_of_death()
    
    data = [current_pop]
    for year in range(YEAR_START, YEAR_END):        
        new_pop = current_pop.copy()
        pop_correct_age = (new_pop.age >= year - YEAR_START) & (new_pop.age < AGE_GROUP_END)
        
        p = all_p[all_p.year == year].reset_index(drop=True)
        new_pop.loc[pop_correct_age, 'population'] *= 1 - p['probability']
        
        new_pop['year'] += 1
        new_pop['age'] += 1
        new_pop.loc[~pop_correct_age, 'population'] = 0
        new_pop.loc[new_pop.age == 111, 'age'] = 0
        
        new_pop = new_pop.sort_values(by=['age']).reset_index(drop=True)
        data.append(new_pop)
        current_pop = new_pop
    return pd.concat(data).sort_values(by=['year', 'age', 'sex']).reset_index(drop=True)


def get_person_years():
    pop = get_population()
    p = get_probability_of_death()
    py = pop.set_index(['year', 'age', 'sex']).population * (1 - p.set_index(['year', 'age', 'sex']).probability)
    py.name = 'person_years'
    return py.reset_index()

def get_yld_rate():
    df = pd.read_csv('/home/james/Downloads/inputs_take2.csv')
    df = df[['age', 'sex', 'pYLD_rate']].rename(columns={'pYLD_rate': 'rate'})
    return df

def get_adjusted_person_years():
    py = get_person_years()
    ylds = get_yld_rate().set_index(['age'])
    
    data = []
    for (sex, year), group in py.groupby(['sex', 'year']):
        group.loc[:, 'person_years'] = (group.set_index(['age']).person_years 
                                        * (1 - ylds[ylds.sex == sex].rate)).values
        data.append(group)
        
    return pd.concat(data, ignore_index=True) 
    
def get_adjusted_life_expectancy():
    py_adj = get_adjusted_person_years().set_index(['age', 'year', 'sex'])
    pop = get_population().set_index(['age', 'year', 'sex'])
    
    ratio = (py_adj.person_years/pop.population).fillna(0)
    ratio.name = 'ratio'
    ratio = ratio.reset_index()
    
    data = []
    for sex in ['male', 'female']:
        ratio_by_sex = ratio[ratio.sex == sex].drop(columns='sex')
        ratio_by_sex = ratio_by_sex.pivot(index='age', columns='year', values='ratio')
        le = pd.DataFrame({'age': range(AGE_GROUP_END + 1),
                           'life_expectancy': 0})
        for age in range(le.age.max()):
            le.loc[le.age == age, 'life_expectancy'] = np.sum(np.diagonal(men, -age))
            
        le['sex'] = sex
        data.append(le)
    return pd.concat(data).reset_index(drop=True)
        
    
        
        

In [42]:
le_adj = get_adjusted_life_expectancy()

/home/james/miniconda3/envs/vivarium/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [46]:
le_adj

,age,life_expectancy,sex
0,0,85.714451,male
1,1,84.739397,male
2,2,83.766116,male
3,3,82.792698,male
4,4,81.819250,male
5,5,80.845774,male
6,6,79.879597,male
7,7,78.913402,male
8,8,77.947187,male
9,9,76.980962,male
